In [ ]:
import sympy
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import symfem
from sympy import Rational
import dill
dill.settings["recurse"] = True
from sympy import Rational as R

In [ ]:
x, y, x_1, y_1, x_2, y_2, x_3, y_3 = sympy.symbols("x y x_1 y_1 x_2 y_2 x_3 y_3")
t = sympy.symbols("t")

In [ ]:
rotation_matrix = sympy.Matrix(
    [
        [+R(0), +R(1)],
        [-R(1), +R(0)]
    ]
)

t1_hat = sympy.Matrix([-R(1), +R(1)]) / sympy.sqrt(2)
t2_hat = sympy.Matrix([+R(0), +R(1)])
t3_hat = sympy.Matrix([+R(1), +R(0)])

n1_hat = rotation_matrix @ t1_hat
n2_hat = rotation_matrix @ t2_hat
n3_hat = rotation_matrix @ t3_hat

### Get basis functions

In [ ]:
monomial_basis = sympy.Matrix(
[1,
 x,
 x**2,
 x**3,
 x**4,
 x**5,
 y,
 x*y,
 x**2*y,
 x**3*y,
 x**4*y,
 y**2,
 x*y**2,
 x**2*y**2,
 x**3*y**2,
 y**3,
 x*y**3,
 x**2*y**3,
 y**4,
 x*y**4,
 y**5]
)

L_4 = R(70)*t**4 - R(140)*t**3 + R(90)*t**2 - R(20)*t + R(1)

def l1(f, n):
    dfdx = f.diff(x)
    dfdy = f.diff(y)
    integrant = L_4 * (n[0]*dfdx + n[1]*dfdy).subs({x: t, y: 1 - t})
    return sympy.integrate(integrant, (t, 0, 1)) 

def l2(f, n):
    dfdx = f.diff(x)
    dfdy = f.diff(y)
    integrant = L_4 * (n[0]*dfdx + n[1]*dfdy).subs({x: 0, y: t})
    return sympy.integrate(integrant, (t, 0, 1))

def l3(f, n):
    dfdx = f.diff(x)
    dfdy = f.diff(y)
    integrant = L_4 * (n[0]*dfdx + n[1]*dfdy).subs({x: t, y: 0})
    return sympy.integrate(integrant, (t, 0, 1)) 

In [394]:
Vander = sympy.Matrix(np.zeros((21, 21)))

for i, basis in enumerate(monomial_basis):

    v1 = {x: 0, y: 0}
    Vander[i, 0] = basis.subs(v1)
    Vander[i, 1] = basis.diff(x).subs(v1)
    Vander[i, 2] = basis.diff(y).subs(v1)
    Vander[i, 3] = basis.diff(x, x).subs(v1)
    Vander[i, 4] = basis.diff(x, y).subs(v1)
    Vander[i, 5] = basis.diff(y, y).subs(v1)

    v2 = {x: 1, y: 0}
    Vander[i, 6] = basis.subs(v2)
    Vander[i, 7] = basis.diff(x).subs(v2)
    Vander[i, 8] = basis.diff(y).subs(v2)
    Vander[i, 9] = basis.diff(x, x).subs(v2)
    Vander[i, 10] = basis.diff(x, y).subs(v2)
    Vander[i, 11] = basis.diff(y, y).subs(v2)

    v3 = {x: 0, y: 1}
    Vander[i, 12] = basis.subs(v3)
    Vander[i, 13] = basis.diff(x).subs(v3)
    Vander[i, 14] = basis.diff(y).subs(v3)
    Vander[i, 15] = basis.diff(x, x).subs(v3)
    Vander[i, 16] = basis.diff(x, y).subs(v3)
    Vander[i, 17] = basis.diff(y, y).subs(v3)

    Vander[i, 18] = l1(basis, n1_hat)
    Vander[i, 19] = l2(basis, n2_hat)
    Vander[i, 20] = l3(basis, n3_hat)

In [396]:
bell_basis = Vander.inv() @ monomial_basis

# Necessary symbols

In [363]:
G1_hat = n1_hat.row_join(t1_hat).T
G2_hat = n2_hat.row_join(t2_hat).T
G3_hat = n3_hat.row_join(t3_hat).T

l1 = ((x_3 - x_2)**2 + (y_3 - y_2)**2) ** R(1, 2)
l2 = ((x_3 - x_1)**2 + (y_3 - y_1)**2) ** R(1, 2)
l3 = ((x_2 - x_1)**2 + (y_2 - y_1)**2) ** R(1, 2)

t1 = sympy.Matrix([x_3 - x_2, y_3 - y_2]) / l1
t2 = sympy.Matrix([x_3 - x_1, y_3 - y_1]) / l2
t3 = sympy.Matrix([x_2 - x_1, y_2 - y_1]) / l3

n1 = rotation_matrix @ t1
n2 = rotation_matrix @ t2
n3 = rotation_matrix @ t3

tau_1 = sympy.Matrix([t1[0]**2, R(2) * t1[0] * t1[1], t1[1]**2])
tau_2 = sympy.Matrix([t2[0]**2, R(2) * t2[0] * t2[1], t2[1]**2])
tau_3 = sympy.Matrix([t3[0]**2, R(2) * t3[0] * t3[1], t3[1]**2])

delta = x_1 * y_2 - x_1 * y_3 - x_2 * y_1 + x_2 * y_3 + x_3 * y_1 - x_3 * y_2

P_1_x = (-y_1 + y_3) / delta
P_1_y = (+x_1 - x_3) / delta

P_2_x = (+y_1 - y_2) / delta
P_2_y = (-x_1 + x_2) / delta

J = sympy.Matrix(
    [
        [P_1_x, P_1_y],
        [P_2_x, P_2_y],
    ]
)

THETA = sympy.Matrix(
    [
        [P_1_x**2, R(2) * P_1_x * P_2_x, P_2_x**2],
        [P_1_y * P_1_x, P_1_y * P_2_x + P_1_x * P_2_y, P_2_x * P_2_y],
        [P_1_y**2, R(2) * P_1_y * P_2_y, P_2_y**2],
    ]
)

G1 = n1.row_join(t1).T
G2 = n2.row_join(t2).T
G3 = n3.row_join(t3).T

B1 = G1_hat @ J.inv().T @ G1.T / l1
B2 = G2_hat @ J.inv().T @ G2.T / l2
B3 = G3_hat @ J.inv().T @ G3.T / l3

right_vals_symbols = sympy.symbols("W1:19")
right_vals = sympy.Matrix(right_vals_symbols)

### Calculate bilinear form and right values for forward pushed reference element

In [ ]:
N = len(bell_basis)

#This is done for fast integration
J11, J12, J21, J22 = sympy.symbols("J11, J12, J21, J22")
J_simp = sympy.Matrix([
    [J11, J12],
    [J21, J22]
])

result_b = [[0 for i in range(N)] for j in range(N)]
result_B = [[0 for i in range(N)] for j in range(N)]
result_A = [[0 for i in range(N)] for j in range(N)]

for idx, jdx in tqdm([(idx, jdx) for idx in range(N) for jdx in range(N)]):
    first = bell_basis[idx]
    second = bell_basis[jdx]

    # -----------------------U------------------------------------------------------------------------------
    u_x = first.diff(x) * J_simp.row(0)[0] + first.diff(y) * J_simp.row(1)[0]
    u_y = first.diff(x) * J_simp.row(0)[1] + first.diff(y) * J_simp.row(1)[1]

    u_xx = u_x.diff(x) * J_simp.row(0)[0] + u_x.diff(y) * J_simp.row(1)[0]
    u_xy = u_x.diff(x) * J_simp.row(0)[1] + u_x.diff(y) * J_simp.row(1)[1]

    u_yy = u_y.diff(x) * J_simp.row(0)[1] + u_y.diff(y) * J_simp.row(1)[1]
    # ------------------------------------------------------------------------------------------------------

    # -----------------------V------------------------------------------------------------------------------
    v_x = second.diff(x) * J_simp.row(0)[0] + second.diff(y) * J_simp.row(1)[0]
    v_y = second.diff(x) * J_simp.row(0)[1] + second.diff(y) * J_simp.row(1)[1]

    v_xx = v_x.diff(x) * J_simp.row(0)[0] + v_x.diff(y) * J_simp.row(1)[0]
    v_xy = v_x.diff(x) * J_simp.row(0)[1] + v_x.diff(y) * J_simp.row(1)[1]

    v_yy = v_y.diff(x) * J_simp.row(0)[1] + v_y.diff(y) * J_simp.row(1)[1]
    # ------------------------------------------------------------------------------------------------------

    # -----------------------bilinear-form------------------------------------------------------------------
    A = u_xx * v_xx
    B = u_xy * v_xy
    b = u_xx * v_yy
    C = u_yy * v_yy
    # ------------------------------------------------------------------------------------------------------

    # -----------------------integrals----------------------------------------------------------------------
    sym_int_B = sympy.integrate(A + R(2) * B + C, (y, 0, 1 - x), (x, 0, 1))
    sym_int_b = sympy.integrate(A + R(2) * b + C, (y, 0, 1 - x), (x, 0, 1))
    sym_int_A = sympy.integrate(first * second, (y, 0, 1 - x), (x, 0, 1))

    result_B[idx][jdx] = sym_int_B
    result_b[idx][jdx] = sym_int_b
    result_A[idx][jdx] = sym_int_A
# -----------------------------------------------------------------------------------------------------

result_b = sympy.Matrix(result_b)
result_B = sympy.Matrix(result_B)
result_A = sympy.Matrix(result_A)

result_A = result_A * abs(J.inv().det())
result_b = result_b * abs(J.inv().det())
result_B = result_B * abs(J.inv().det())

In [ ]:
N = len(bell_basis)
result_C = [0 for j in range(N)]

for idx in tqdm(range(N)):
    u = bell_basis[idx]
    sym_int_C = sympy.integrate(u, (y, 0, 1 - x), (x, 0, 1))
    result_C[idx] = sym_int_C

result_C = sympy.Matrix(result_C)
result_C = result_C * abs(J.inv().det())

In [ ]:
result_b = result_b.subs({
        J11: J[0, 0],
        J12: J[0, 1],
        J21: J[1, 0],
        J22: J[1, 1]
})

result_B = result_B.subs({
        J11: J[0, 0],
        J12: J[0, 1],
        J21: J[1, 0],
        J22: J[1, 1]
})

### Save the results

In [ ]:
dill.dump(result_b, open("../calculations/pushed_forward_result_for_b_bell", "wb"))
dill.dump(result_B, open("../calculations/pushed_forward_result_for_B_bell", "wb"))
dill.dump(result_A, open("../calculations/pushed_forward_result_for_A_bell", "wb"))
dill.dump(result_C, open("../calculations/pushed_forward_result_for_C_bell", "wb"))

### Load the results

In [ ]:
result_b = dill.load(open("../calculations/pushed_forward_result_for_b_bell", "rb"))
result_B = dill.load(open("../calculations/pushed_forward_result_for_B_bell", "rb"))
result_A = dill.load(open("../calculations/pushed_forward_result_for_A_bell", "rb"))
result_C = dill.load(open("../calculations/pushed_forward_result_for_C_bell", "rb"))

# Define transformation matrices

### $V^{c}$

In [305]:
THETA_s = sympy.simplify(THETA * delta**2)
THETA_inv = THETA_s.inv() * delta**2

In [372]:
V_c = sympy.diag(
    R(1),
    J.inv().T,
    THETA_inv,
    R(1),
    J.inv().T,
    THETA_inv,
    R(1),
    J.inv().T,
    THETA_inv,
    B1,
    B2,
    B3,
)

### $E$

In [373]:
E = np.zeros((21, 24), dtype=int)
for i in range(21):
    for j in range(24):
        if (i <= 18) and (j <= 18) and (i == j):
            E[i][j] = 1
        elif (i == 19) and (j == 20):
            E[i][j] = 1
        elif (i == 20) and (j == 22):
            E[i][j] = 1
E = sympy.Matrix(E)

### $D$

In [375]:
D = sympy.Matrix(
    [
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
        [
            0,
            0,
            0,
            0,
            0,
            0,
            -R(1, 21),
            -R(1, 42) * l1 * t1[0],
            -R(1, 42) * l1 * t1[1],
            -R(1, 252) * l1**2 * tau_1[0],
            -R(1, 252) * l1**2 * tau_1[1],
            -R(1, 252) * l1**2 * tau_1[2],
            +R(1, 21),
            -R(1, 42) * l1 * t1[0],
            -R(1, 42) * l1 * t1[1],
            +R(1, 252) * l1**2 * tau_1[0],
            +R(1, 252) * l1**2 * tau_1[1],
            +R(1, 252) * l1**2 * tau_1[2],
            0,
            0,
            0,
        ],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
        [
            -R(1, 21),
            -R(1, 42) * l2 * t2[0],
            -R(1, 42) * l2 * t2[1],
            -R(1, 252) * l2**2 * tau_2[0],
            -R(1, 252) * l2**2 * tau_2[1],
            -R(1, 252) * l2**2 * tau_2[2],
            0,
            0,
            0,
            0,
            0,
            0,
            +R(1, 21),
            -R(1, 42) * l2 * t2[0],
            -R(1, 42) * l2 * t2[1],
            +R(1, 252) * l2**2 * tau_2[0],
            +R(1, 252) * l2**2 * tau_2[1],
            +R(1, 252) * l2**2 * tau_2[2],
            0,
            0,
            0,
        ],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
        [
            -R(1, 21),
            -R(1, 42) * l3 * t3[0],
            -R(1, 42) * l3 * t3[1],
            -R(1, 252) * l3**2 * tau_3[0],
            -R(1, 252) * l3**2 * tau_3[1],
            -R(1, 252) * l3**2 * tau_3[2],
            +R(1, 21),
            -R(1, 42) * l3 * t3[0],
            -R(1, 42) * l3 * t3[1],
            +R(1, 252) * l3**2 * tau_3[0],
            +R(1, 252) * l3**2 * tau_3[1],
            +R(1, 252) * l3**2 * tau_3[2],
            0,
            0,
            0,
            0,
            0,
            0,
            0,
            0,
            0,
        ],
    ]
)

### $M$

In [377]:
V = E @ V_c @ D
M_c = V.T

In [378]:
E_2 = np.zeros((18, 21), dtype=int)
for i in range(18):
    for j in range(21):
        if (i == j):
            E_2[i][j] = 1
E_2 = sympy.Matrix(E_2)

In [379]:
M = E_2 @ M_c

### Apply transformations

In [201]:
bilinear_form_biharmonic_without_mixed = M @ result_b @ M.T

In [202]:
bilinear_form_biharmonic_with_mixed = M @ result_B @ M.T

In [ ]:
right_part = M @ result_A @ M.T @ right_vals

In [ ]:
integral_values = M @ result_C

In [ ]:
final_parameters = [x_1, y_1, x_2, y_2, x_3, y_3]

In [417]:
bilinear_form_biharmonic_without_mixed_f = sympy.lambdify(
    final_parameters, 
    bilinear_form_biharmonic_without_mixed,
    cse=True
)

In [418]:
bilinear_form_biharmonic_with_mixed_f = sympy.lambdify(
    final_parameters, 
    bilinear_form_biharmonic_with_mixed,
    cse=True
)

In [419]:
right_part_f = sympy.lambdify(
    [*final_parameters, *right_vals_symbols],
    right_part, 
    cse=True
)

In [420]:
integral_values_f = sympy.lambdify(
    final_parameters,
    integral_values, 
    cse=True
)

### Save the results

In [421]:
dill.dump(
    bilinear_form_biharmonic_without_mixed_f,
    open("../calculations/bell_quintic_biharmonic_matrix_without_mixed_simplified", "wb"),
)
dill.dump(
    bilinear_form_biharmonic_with_mixed_f,
    open("../calculations/bell_quintic_biharmonic_matrix_with_mixed_simplified", "wb"),
)
dill.dump(
    right_part_f,
    open("../calculations/bell_quintic_biharmonic_matrix_right_values_simplified", "wb"),
)
dill.dump(
    integral_values_f,
    open("../calculations/bell_quintic_biharmonic_matrix_integral_values_simplified", "wb"),
)